## course  (2)  Tuning

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import lightgbm as lgb
import optuna.integration.lightgbm as lgb
from sklearn import datasets
from sklearn.model_selection import train_test_split
import gc
%matplotlib inline
pd.set_option('display.max_Columns', 100)

In [2]:
ALL_MERGE = 'intermediate/all_merge_12.f'

In [3]:
all_pitch = pd.read_feather(ALL_MERGE)
all_pitch.shape

(778767, 171)

### 不要な列を削除

In [4]:
all_pitch.drop(
    columns=[
        '年度', 
        '試合ID', 
        'ホームチームID', 'アウェイチームID', 
        '投手ID', '投手チームID', 
        '打者ID', '打者チームID', 
        'プレイ前走者状況', 
        '捕手ID', 
        'opening_date', 'game_date',
        'start_time', 'game_time', 'elapsed_time'
    ], inplace=True)

### Rename

In [5]:
all_pitch.rename(columns={
    'データ内連番': 'No',
    '投球位置区域': 'pitch_area',
    '試合内連番': 'No_in_game',
    '試合内投球数': 'pitch_cnt_in_game',
    'イニング': 'inning',
    'イニング内打席数': 'bat_cnt_in_inning',
    '打席内投球数': 'pitch_cnt_in_bat',
    '投手登板順': 'pitch_order',
    '投手試合内対戦打者数': 'player_cnt_in_game',
    '投手試合内投球数': 'pitcher_cnt_in_game',
    '投手イニング内投球数': 'pitcher_cnt_in_inning',
    '打者打順': 'bat_order',
    '打者試合内打席数': 'bat_cnt_in_game',
    'プレイ前ホームチーム得点数': 'home_point',
    'プレイ前アウェイチーム得点数': 'away_point',
    'プレイ前アウト数': 'out_cnt',
    'プレイ前ボール数': 'ball_cnt',
    'プレイ前ストライク数': 'strike_cnt',
}, inplace=True)

In [6]:
print(all_pitch.shape)
all_pitch.head()

(778767, 156)


,No,ball,pitch_area,No_in_game,pitch_cnt_in_game,inning,bat_cnt_in_inning,pitch_cnt_in_bat,pitch_order,player_cnt_in_game,pitcher_cnt_in_game,pitcher_cnt_in_inning,bat_order,bat_cnt_in_game,home_point,away_point,out_cnt,ball_cnt,strike_cnt,bc_straight,bc_curve,bc_slider,bc_shoot,bc_fork,bc_changeup,bc_sinker,bc_cutball,bc_area0,bc_area1,bc_area2,bc_area3,bc_area4,bc_area5,bc_area6,bc_area7,bc_area8,bc_area9,bc_area10,bc_area11,bc_area12,first,second,third,base_cnt,pitch_LR,bat_LR,top_bot,role,pos_pit,date_from_opening,...,firm_bat,height_bat,weight_bat,draft_order_bat,salary_bat,foreigner_bat,batter_cnt_bat,bat_game_cnt_bat,company_bat,univ_bat,highsch_bat,age_bat,play_year_bat,salary_year_bat,salary_x_year_bat,bmi_bat,firm,height,weight,draft_order,salary,foreigner,batter_cnt,bat_game_cnt,company,univ,highsch,age,play_year,salary_year,salary_x_year,bmi,salary_dif_p-b,play_year_dif_p-b,age_dif_p-b,salary_year_dif_p-b,salary_x_year_dif_p-b,bmi_dif_p-b,salary_dif_p-c,play_year_dif_p-c,age_dif_p-c,salary_year_dif_p-c,salary_x_year_dif_p-c,bmi_dif_p-c,salary_dif_b-c,play_year_dif_b-c,age_dif_b-c,salary_year_dif_b-c,salary_x_year_dif_b-c,bmi_dif_b-c
0,0,0.0,0.0,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0.477023,0.097592,0.200870,0.074454,0.046799,0.041882,0.008619,0.052760,0.048489,0.071105,0.059259,0.047675,0.063792,0.049150,0.046338,0.070075,0.059013,0.117934,0.090325,0.114800,0.162045,0,0,0,0,0.0,1.0,1,1,0,0,...,0,176,74,5.0,15000,0,523.0,119.0,1,1,0,32,8.0,1875.0,120000.0,23.889463,0,178,83,1.0,5000,0,443.0,135.0,1,1,0,28,4.0,1250.0,20000.0,26.196187,9000,-2.0,-3,2125.0,24000.0,2.141358,19000,2.0,1,2750.0,124000.0,-0.165367,10000,4.0,4,625.0,100000.0,-2.306725
1,1,0.0,8.0,2,2,1,1,2,1,1,2,2,1,1,0,0,0,0,1,0.459832,0.090366,0.161893,0.068313,0.099045,0.061079,0.012505,0.046967,0.035008,0.056450,0.047706,0.035812,0.044074,0.034687,0.037484,0.058379,0.050407,0.112451,0.103867,0.161700,0.221976,0,0,0,0,0.0,1.0,1,1,0,0,...,0,176,74,5.0,15000,0,523.0,119.0,1,1,0,32,8.0,1875.0,120000.0,23.889463,0,178,83,1.0,5000,0,443.0,135.0,1,1,0,28,4.0,1250.0,20000.0,26.196187,9000,-2.0,-3,2125.0,24000.0,2.141358,19000,2.0,1,2750.0,124000.0,-0.165367,10000,4.0,4,625.0,100000.0,-2.306725
2,2,0.0,5.0,3,3,1,1,3,1,1,3,3,1,1,0,0,0,0,2,0.469730,0.066062,0.159395,0.031361,0.162940,0.062722,0.014112,0.033679,0.021884,0.035656,0.030884,0.020316,0.021748,0.021816,0.028429,0.036678,0.029520,0.133761,0.131170,0.201595,0.286542,0,0,0,0,0.0,1.0,1,1,0,0,...,0,176,74,5.0,15000,0,523.0,119.0,1,1,0,32,8.0,1875.0,120000.0,23.889463,0,178,83,1.0,5000,0,443.0,135.0,1,1,0,28,4.0,1250.0,20000.0,26.196187,9000,-2.0,-3,2125.0,24000.0,2.141358,19000,2.0,1,2750.0,124000.0,-0.165367,10000,4.0,4,625.0,100000.0,-2.306725
3,3,0.0,12.0,4,4,1,2,1,1,2,4,4,2,1,0,0,1,0,0,0.477023,0.097592,0.200870,0.074454,0.046799,0.041882,0.008619,0.052760,0.048489,0.071105,0.059259,0.047675,0.063792,0.049150,0.046338,0.070075,0.059013,0.117934,0.090325,0.114800,0.162045,0,0,0,0,0.0,0.0,1,1,0,0,...,0,180,74,1.0,7040,0,268.0,81.0,0,0,1,40,22.0,320.0,154880.0,22.839506,0,178,83,1.0,5000,0,443.0,135.0,1,1,0,28,4.0,1250.0,20000.0,26.196187,16960,-16.0,-11,3680.0,-10880.0,3.191314,19000,2.0,1,2750.0,124000.0,-0.165367,2040,18.0,12,-930.0,134880.0,-3.356681
4,4,0.0,8.0,5,5,1,2,2,1,2,5,5,2,1,0,0,1,0,1,0.459832,0.090366,0.161893,0.068313,0.099045,0.061079,0.012505,0.046967,0.035008,0.056450,0.047706,0.035812,0.044074,0.034687,0.037484,0.058379,0.050407,0.112451,0.103867,0.161700,0.221976,0,0,0,0,0.0,0.0,1,1,0,0,...,0,180,74,1.0,7040,0,268.0,81.0,0,0,1,40,22.0,320.0,154880.0,22.839506,0,178,83,1.0,5000,0,443.0,135.0,1,1,0,28,4.0,1250.0,20000.0,26.196187,16960,-16.0,-11,3680.0,-10880.0,3.191314,19000,2.0,1,2750.0,124000.0,-0.165367,2040,18.0,12,-930.0,134880.0,-3.356681


### train
- 行数: 257117

In [7]:
train = all_pitch.dropna(subset=['pitch_area'])
train.shape

(257117, 156)

### test
- 行数: 521650

In [8]:
test = all_pitch[all_pitch['pitch_area'].isnull()]
test.shape

(521650, 156)

In [9]:
del all_pitch
gc.collect()

60

In [10]:
train_d = train.drop([
    'No', 
    'pitch_area', 
    'ball'
], axis=1)

In [11]:
test_d = test.drop([
    'No', 
    'pitch_area', 
    'ball'
], axis=1)

## Dataset作成

In [12]:
X_train, X_test, y_train, y_test = train_test_split(train_d, train['pitch_area'])
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

### LGBM学習
- multiclass : クラス数=8

In [13]:
lgb_param = {
        'objective' : 'multiclass',
        'boosting_type': 'gbdt',
        'metric' : 'multi_logloss',
        'num_class' : 13,
    }

### Tuningしながら学習

In [14]:
%%time
best_params, tuning_history = dict(), list()
lgb_model = lgb.train(lgb_param, lgb_train,
                      valid_sets=lgb_eval,
                      verbose_eval=0,
                      best_params=best_params,
                      tuning_history=tuning_history)
print('Best Params:', best_params)
#print('Tuning history:', tuning_history)

/home/yasunobu_umehara/anaconda3/lib/python3.7/site-packages/optuna/_experimental.py:90: ExperimentalWarning: train is experimental (supported from v0.18.0). The interface can change in the future.
  ExperimentalWarning,
tune_num_leaves, val_score: 2.342836: 100%|##########| 20/20 [1:11:53<00:00, 215.67s/it]
tune_bagging_fraction_and_bagging_freq, val_score: 2.342365:  10%|#         | 1/10 [02:11<19:44, 131.56s/it][I 2020-05-07 05:29:46,018] Finished trial#0 with value: 2.3423651502924683 with parameters: {'bagging_fraction': 0.937314013173315, 'bagging_freq': 6}. Best is trial#0 with value: 2.3423651502924683.
tune_bagging_fraction_and_bagging_freq, val_score: 2.342359:  20%|##        | 2/10 [04:31<17:53, 134.17s/it][I 2020-05-07 05:32:06,305] Finished trial#1 with value: 2.342359424494063 with parameters: {'bagging_fraction': 0.9789957630836268, 'bagging_freq': 3}. Best is trial#1 with value: 2.342359424494063.
tune_bagging_fraction_and_bagging_freq, val_score: 2.342048:  30%|###    

tune_bagging_fraction_and_bagging_freq, val_score: 2.341989:  40%|####      | 4/10 [08:45<13:00, 130.11s/it][I 2020-05-07 05:36:19,808] Finished trial#3 with value: 2.3419886453005607 with parameters: {'bagging_fraction': 0.8391111798378441, 'bagging_freq': 4}. Best is trial#3 with value: 2.3419886453005607.
tune_bagging_fraction_and_bagging_freq, val_score: 2.341989:  50%|#####     | 5/10 [10:21<09:59, 119.89s/it][I 2020-05-07 05:37:55,857] Finished trial#4 with value: 2.3445074227132072 with parameters: {'bagging_fraction': 0.5231732160710658, 'bagging_freq': 2}. Best is trial#3 with value: 2.3419886453005607.
tune_bagging_fraction_and_bagging_freq, val_score: 2.341989:  60%|######    | 6/10 [12:10<07:46, 116.66s/it][I 2020-05-07 05:39:44,973] Finished trial#5 with value: 2.3425832763529906 with parameters: {'bagging_fraction': 0.6891014296370164, 'bagging_freq': 6}. Best is trial#3 with value: 2.3419886453005607.
tune_bagging_fraction_and_bagging_freq, val_score: 2.341989:  70%|####

tune_lambda_l1_and_lambda_l2, val_score: 2.339350:  85%|########5 | 17/20 [35:44<06:27, 129.11s/it][I 2020-05-07 06:29:04,396] Finished trial#16 with value: 2.3417639527876584 with parameters: {'lambda_l1': 0.0007023936286736025, 'lambda_l2': 2.5777088797080187e-05}. Best is trial#15 with value: 2.3393503448196284.
tune_lambda_l1_and_lambda_l2, val_score: 2.339350:  90%|######### | 18/20 [37:57<04:20, 130.13s/it][I 2020-05-07 06:31:16,928] Finished trial#17 with value: 2.3396925013700187 with parameters: {'lambda_l1': 4.697381145201627, 'lambda_l2': 2.0769970649084306e-05}. Best is trial#15 with value: 2.3393503448196284.
tune_lambda_l1_and_lambda_l2, val_score: 2.339350:  95%|#########5| 19/20 [40:14<02:12, 132.16s/it][I 2020-05-07 06:33:33,824] Finished trial#18 with value: 2.3394233576292205 with parameters: {'lambda_l1': 9.667560032874993, 'lambda_l2': 1.912961009452857e-05}. Best is trial#15 with value: 2.3393503448196284.
tune_lambda_l1_and_lambda_l2, val_score: 2.339350: 100%|##

Best Params: {'lambda_l1': 8.769293390201968, 'lambda_l2': 3.913949617576324e-05, 'num_leaves': 6, 'feature_fraction': 0.4, 'bagging_fraction': 0.8391111798378441, 'bagging_freq': 4, 'min_child_samples': 50}
CPU times: user 23h 39min 57s, sys: 4min 27s, total: 23h 44min 25s
Wall time: 2h 58min 40s


### Feature Importance

In [15]:
fi = lgb_model.feature_importance()
fn = lgb_model.feature_name()
df_feature_importance = pd.DataFrame({'feat_name':fn, 'feat_imp':fi})
df_feature_importance.sort_values('feat_imp', inplace=True)
df_feature_importance.tail(30)

,feat_name,feat_imp
85,changeup_pit,781
141,salary_dif_p-c,789
145,salary_x_year_dif_p-c,809
105,weight_bat,836
100,salary_year_pit,851
81,curve_pit,855
147,salary_dif_b-c,872
135,salary_dif_p-b,882
102,bmi_pit,889
116,salary_year_bat,890


In [16]:
%%time
predict = lgb_model.predict(test_d, num_iteration = lgb_model.best_iteration)

CPU times: user 6min 9s, sys: 572 ms, total: 6min 10s
Wall time: 47.6 s


In [17]:
predict

array([[0.05934499, 0.12851578, 0.06894903, ..., 0.07022527, 0.08668852,
        0.09782472],
       [0.05128317, 0.12085568, 0.08107382, ..., 0.06291674, 0.10315632,
        0.09952474],
       [0.03182537, 0.08144164, 0.06080754, ..., 0.09260252, 0.19655931,
        0.12252716],
       ...,
       [0.03525054, 0.03914734, 0.0420999 , ..., 0.15628268, 0.22210111,
        0.17701973],
       [0.01970621, 0.02632608, 0.02924493, ..., 0.16067407, 0.28334237,
        0.16335488],
       [0.03156732, 0.03473612, 0.03852846, ..., 0.1369145 , 0.26259794,
        0.18851998]])

## 結果出力

In [18]:
submit = pd.DataFrame(predict)
submit.reset_index(inplace=True)
print(submit.shape)
submit.head()

(521650, 14)


,index,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0,0.059345,0.128516,0.068949,0.046305,0.058436,0.027611,0.055792,0.060319,0.039664,0.200326,0.070225,0.086689,0.097825
1,1,0.051283,0.120856,0.081074,0.044444,0.056770,0.043996,0.041929,0.064163,0.044930,0.184957,0.062917,0.103156,0.099525
2,2,0.031825,0.081442,0.060808,0.030913,0.038031,0.033983,0.037946,0.052414,0.032788,0.188162,0.092603,0.196559,0.122527
3,3,0.032404,0.101843,0.066963,0.033110,0.043111,0.032378,0.043633,0.059413,0.040175,0.220013,0.080928,0.125233,0.120797
4,4,0.037727,0.118945,0.074767,0.036156,0.047695,0.040911,0.047059,0.060557,0.042301,0.184626,0.078385,0.113043,0.117828


### 2017の平均
- ストレート           0.468254
- カーブ             0.074725
- スライダー           0.185806
- シュート            0.070633
- フォーク            0.083013
- チェンジアップ         0.054217
- シンカー            0.011361
- カットボール          0.051992

In [19]:
submit.mean()

index    260824.500000
0             0.048884
1             0.069330
2             0.056590
3             0.046819
4             0.061129
5             0.046750
6             0.043212
7             0.065747
8             0.053501
9             0.108827
10            0.087440
11            0.133672
12            0.178098
dtype: float64

In [20]:
#submit.to_csv('submit/tuning.csv', header=False, index=False)

### Course Tuning結果(12)
`
Best Params: {
    'lambda_l1': 8.769293390201968, 
    'lambda_l2': 3.913949617576324e-05, 
    'num_leaves': 6, 
    'feature_fraction': 0.4, 
    'bagging_fraction': 0.8391111798378441, 
    'bagging_freq': 4, 
    'min_child_samples': 50
}
`
Wall time: 2h 58min 40s